In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import sys
sys.path.insert(0, '../AutoDiff')
import variables as v
import AD_numpy as anp
import vector_variables as vv
sys.path.insert(0, '../Implementation')
import Optimizer as op

# An Optimization Package with AD functionalities

## Motivation

### Motivation

Taking derivatives is an essential operation in numerical methods, optimization, and science. From a computational perspective, however, calculating a derivative can be difficult.

**Finite Differences** requires careful selection of $\epsilon$

**Symbolic Differentiation** is infeasible for complicated functions, especially for higher order derivatives

### Motivation

**Automatic Differentiation** overcomes these challenges by providing both quick and accurate derivatives.

This also allows us to provide optimization methods that are fast and accurate. Most of the standard optimization modules such as `scipy.optimize` do not rely on automatic differentiation. 

### Brief Mathematical Background 

...

### Demo for AD module

In [2]:
# user has a given function
f1 = lambda x, y: anp.exp(3*x) + anp.log(y/x)

In [3]:
# he can calculate its value
f1(2, 3)

403.83425860084327

In [4]:
# but now he wishes to get it' derivative
a = v.Variable('a', 2)
b = v.Variable('b', 3)
res1 = f1(a,b)

In [5]:
print('Value: {}.'.format(res1.val))
print('Partial Derivative wrt a: {}'.format(res1.partial_der(a)))
print('Jacobian: {}'.format(res1.jacobian()))

Value: 403.83425860084327.
Partial Derivative wrt a: 1209.7863804782053
Jacobian: {'b': 0.3333333333333333, 'a': 1209.7863804782053}


In [6]:
# similarly, if he has a vector function
@vv.vectorize_variable
def vec_fn(x, y, z):
    f1 = anp.cos(x) + anp.sin(y)+ anp.cos(z) 
    f2 = x**2 - y**2 - z**2
    return np.array([f1,f2])

In [7]:
# he can extract the values, jacobian, and partial derivatives similarly
c = v.Variable('c', 5)
res2 = vec_fn(a, b, c)

In [12]:
print('Value: {}.'.format(res2.val))
print('Partial Derivative wrt a: {}'.format(res2.partial_der(a)))
display(res2.jacobian())

Value: [ 8.63535698e-03 -3.00000000e+01].
Partial Derivative wrt a: [-0.90929743  4.        ]


,a,b,c
0,-0.909297,-0.989992,0.958924
1,4.000000,-6.000000,-10.000000


### Now all that seems trivial. 

When would a user just want to solely calculate the gradient of a function?

#### Instead, suppose a user wants to find the root of a given function using Newton's method

In [26]:
def newton_method_scalar(fn, x, threshold, max_iter, verbose=True, norm=2):
    
    var_names = ['x'+str(idx) for idx in range(len(x))]
    
    x = np.array(x)
    nums_iteration = 0
    while True:
        x_new = x - fn(*x) / _get_grad(fn, x, var_names)

        # print iteration output
        if verbose is True:
            print(f'Iteration at {nums_iteration}, at {x} ')
        
        # threshold stopping condition 
        if np.linalg.norm(x-x_new, norm) < threshold:
            print(f'After {nums_iteration} iterations, found a root: {x_new}')
            break
        
        # iteration stopping condition
        if nums_iteration >= max_iter:
            break
        nums_iteration +=1
        x = x_new
        
def _get_grad(fn, x, var_names):
    variables = [v.Variable(var_names[idx], x_n) for idx, x_n in enumerate(x)]
    out = fn(*variables)
    jacobian = out.jacobian()
    grad = np.array([jacobian[name] for name in var_names])
    return grad

In [27]:
f2 = lambda x, y, z: (x-4)**2 + (y-3)**2 + (z-2)**2
newton_method_scalar(f2, [3, 2, 5], 1e-6, 50, verbose=True)

Iteration at 0, at [3 2 5] 
Iteration at 1, at [8.5        7.5        3.16666667] 
Iteration at 2, at [  3.84876543   2.84876543 -14.77380952] 
Iteration at 3, at [934.21288005 933.21288005  -6.38554121] 
Iteration at 4, at [3.96220365e+00 2.96220365e+00 1.03186854e+05] 
Iteration at 5, at [1.40848446e+11 1.40848446e+11 5.15944269e+04] 
Iteration at 6, at [ 3.99053955e+00  2.99053955e+00 -3.84519315e+17] 
Iteration at 7, at [ 7.81438071e+36  7.81438071e+36 -1.92259658e+17] 
Iteration at 8, at [0.00000000e+00 0.00000000e+00 3.17614973e+56] 
Iteration at 9, at [1.26099089e+112 1.68132118e+112 1.58807486e+056] 
Iteration at 10, at [-4.90385345e+111  3.67789009e+111 -1.39065829e+168] 
Iteration at 11, at [ inf -inf  inf] 
Iteration at 12, at [nan nan nan] 
Iteration at 13, at [nan nan nan] 
Iteration at 14, at [nan nan nan] 
Iteration at 15, at [nan nan nan] 
Iteration at 16, at [nan nan nan] 
Iteration at 17, at [nan nan nan] 
Iteration at 18, at [nan nan nan] 
Iteration at 19, at [nan na

C:\Users\shengsiong\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in double_scalars
  """Entry point for launching an IPython kernel.
../AutoDiff\variables.py:131: RuntimeWarning: overflow encountered in double_scalars
  pow = binary_user_function(lambda x,y: x**y, lambda x,y: y*(x**(y-1)), lambda x,y: x**y*np.log(x))
C:\Users\shengsiong\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
